In [1]:
from logging import error, exception
# from httplib2.error import FailedToDecompressContent
import pandas as pd
import pygsheets
import os
from datetime import datetime, date
import sys

import gsheet_utility as gsheetUtility
import Email_Code as fkEmail
import data_processing as data_processing
import numpy as np

In [2]:
basePath = '/Users/a/Documents/GitHub/exception_visibility/'
date_master_file = pd.read_csv( '/Users/a/Documents/GitHub/exception_visibility/Dashboard/data/week_details.csv')
date_master_file['weekday'] = pd.to_datetime(date_master_file["weekday"],format = '%d/%m/%Y').dt.date
date_master_file['month_year'] = pd.to_datetime(date_master_file["month_year"],format = '%d/%m/%Y').dt.date

rc_raw_data_location = basePath + 'ETL/RC_Raw_Data/'
mh_raw_data_location = basePath + 'ETL/MH_Raw_Data/'
orphan_raw_data_location = basePath + 'ETL/Orphan_Raw_Data/'
hv_orphan_raw_data_location = basePath + 'ETL/HV_Orphan_Raw_Data/'
logistics_raw_data_location = basePath + 'ETL/Logistics_Raw_Data/'

today = date.today()
today = str(today.year) + "-" + str('{:02d}'.format(today.month)) + "-" + str('{:02d}'.format(today.day))

rcgsheetdatapath = basePath + 'ETL/gsheet_data/rc_data/' + today + '.csv'
mhgsheetdatapath = basePath + 'ETL/gsheet_data/mh_data/' + today + '.csv'
orphangsheetdatapath = basePath + 'ETL/gsheet_data/orphan_data/' + today + '.csv'
hv_orphangsheetdatapath = basePath + 'ETL/gsheet_data/hv_orphan_data/' + today + '.csv'
logisticsgsheetdatapath = basePath + 'ETL/gsheet_data/hv_orphan_data/' + today + '.csv'

error_message = ''
html_message = ''

In [3]:
html_message = html_message + "Data Ingestion Job for Logistics started at : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Starting Logistics data process')
logisticsData = gsheetUtility.get_gsheet_data('https://docs.google.com/spreadsheets/d/1beqaUyvM7gb8qVnl9sQdQokzKtuftfulAmqmr72IxI0/edit?usp=sharing', 'A2', logisticsgsheetdatapath,'logistics')

logisticsdbcolumns = ['scanned_timestamp','email_address','zone','asset','hub_name','orphan_type','orphan_id','super_category','product_title','product_value','motherhub_name']
html_message = html_message + "completed fetching Logistics orphan raw data from google sheet : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Fetching Logistics orphan data process Completed')
logisticsData.columns=logisticsdbcolumns
# logisticsData = gsheetUtility.assignDBColumns(logisticsData, gsheet_asset=logisticsData['asset'], dbColumns=logisticsdbcolumns)
logisticsData = data_processing.datatype_conversion(logisticsData, 'scanned_timestamp', 'datetime')
logisticsData['scanned_date'] = logisticsData['scanned_timestamp'].dt.date
logisticsData = logisticsData[logisticsData.scanned_date >= pd.to_datetime('2021-10-01')]
logisticsData = logisticsData[logisticsData.asset.isin(['Last Mile','First Mile'])]
logisticsData = data_processing.fetch_date_details(logisticsData, 'exception_log_timestamp', date_master_file)

print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Starting to create Logistics orphan raw data files')
html_message = html_message + "Starting to create Logistics orphan raw data files at : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'

logisticsData.orphan_id = logisticsData.orphan_id.str.upper()
filenames = []
created_raw_files = data_processing.fetch_created_files(logistics_raw_data_location)
filenames = data_processing.create_rewrite_raw_files(logisticsData,'scanned_date', created_raw_files, logistics_raw_data_location )
html_message = html_message + "Completed creating Logistics orphan raw data files at : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Completed creating Logistics orphan raw data files')

print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Starting to collate data for Logistics dashboard')
html_message = html_message + "Starting to create Logistics orphan raw data files at : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
logistics_final_columns = ['scanned_timestamp','email_address','zone','asset','hub_name','orphan_type','orphan_id','super_category','product_title','product_value', 
                        'scanned_date', 'weekend', 'month', 'year', 'month_year','weeknum']
created_raw_files = data_processing.fetch_created_files(logistics_raw_data_location)
logistics_dashboard_data = data_processing.collate_data_for_dashboard(datetime.now().date(), 180, created_raw_files, logistics_raw_data_location, logistics_final_columns)
logistics_dashboard_data.to_csv(basePath + 'Dashboard/data/logistcs_orphan_full_data.csv', index=False)

12/23/2021 09:16:12 Starting Logistics data process
12/23/2021, 09:16:12 Starting Fetching Gsheet Data
12/23/2021, 09:16:25 Fetching Gsheet Completed
logistics
A2
12/23/2021 09:16:26 Fetching Logistics orphan data process Completed
Converting scanned_timestamp to datetime
Converted scanned_timestamp to datetime
mapping dates and weekend to scanned date
12/23/2021 09:17:02 Starting to create Logistics orphan raw data files
fetching names of already created raw files
started creating raw files
creating raw file for the date 2021-10-04.csv
creating raw file for the date 2021-10-05.csv
creating raw file for the date 2021-10-06.csv
creating raw file for the date 2021-10-07.csv
creating raw file for the date 2021-10-08.csv
creating raw file for the date 2021-10-09.csv
creating raw file for the date 2021-10-10.csv
creating raw file for the date 2021-10-11.csv
creating raw file for the date 2021-10-12.csv
creating raw file for the date 2021-10-13.csv
creating raw file for the date 2021-10-14.c

In [ ]:
logisticsData

In [4]:
orpdf = pd.read_csv('/Users/a/Documents/GitHub/exception_visibility/Dashboard/data/orphan_full_data.csv', low_memory=False)
hvdf = pd.read_csv('/Users/a/Documents/GitHub/exception_visibility/Dashboard/data/hv_orphan_full_data.csv', low_memory=False)
logisticsdf = pd.read_csv('/Users/a/Documents/GitHub/exception_visibility/Dashboard/data/logistcs_orphan_full_data.csv', low_memory=False)
logisticsdf = pd.read_csv('/Users/a/Documents/GitHub/exception_visibility/Dashboard/data/logistcs_orphan_full_data.csv', low_memory=False)
rcdf = pd.read_csv('/Users/a/Documents/GitHub/exception_visibility/Dashboard/data/rc_full_data.csv', low_memory=False)

In [ ]:
date_agg='month_year'
orpdf['month_year'] = pd.to_datetime(orpdf.scanned_date).dt.strftime('%b') + '-' + pd.to_datetime(orpdf.scanned_date).dt.year.astype(str)
logisticsdf['month_year'] = pd.to_datetime(logisticsdf.scanned_date).dt.strftime('%b') + '-' + pd.to_datetime(logisticsdf.scanned_date).dt.year.astype(str)
hvdf['month_year'] = pd.to_datetime(hvdf.scanned_date).dt.strftime('%b') + '-' + pd.to_datetime(hvdf.scanned_date).dt.year.astype(str)

month_year = ['Jan-2021','Feb-2021', 'Mar-2021','Apr-2021','May-2021','Jun-2021','Jul-2021','Aug-2021','Sep-2021','Oct-2021','Nov-2021','Dec-2021',
                'Jan-2022','Feb-2022', 'Mar-2022','Apr-2022','May-2022','Jun-2022','Jul-2022','Aug-2022','Sep-2022','Oct-2022','Nov-2022','Dec-2022',
                'Jan-2023','Feb-2023', 'Mar-2023','Apr-2023','May-2023','Jun-2023','Jul-2023','Aug-2023','Sep-2023','Oct-2023','Nov-2023','Dec-2023']
# date_agg = 'year'
if date_agg=='month_yr':
    date_agg='month_year'

orphan_pivot = pd.DataFrame(pd.pivot_table(orpdf, index='asset', columns=[date_agg], values=['scanned_timestamp'], aggfunc=len)).reset_index().rename(columns={'scanned_timestamp': 'orphan_count'})
hv_pivot = pd.DataFrame(pd.pivot_table(hvdf, index='asset', columns=[date_agg], values=['scanned_timestamp'], aggfunc=len)).reset_index().rename(columns={'scanned_timestamp': 'orphan_count'})
logistics_pivot = pd.DataFrame(pd.pivot_table(logisticsdf, index='asset', columns=[date_agg], values=['scanned_timestamp'], aggfunc=len)).reset_index().rename(columns={'scanned_timestamp': 'orphan_count'})

overall_orphans_generated = orphan_pivot.append(logistics_pivot).append(hv_pivot).fillna('0')
overall_orphans_generated.columns = overall_orphans_generated.columns.droplevel(0)
overall_orphans_generated.rename(columns={'':'asset'}, inplace=True)
if date_agg=='month_year':    
    table_columns = [x for x in month_year if x in overall_orphans_generated.columns.tolist()]
    table_columns.insert(0,'asset')
    overall_orphans_generated[table_columns].reset_index().drop(columns='index')


In [13]:
lastmile_df = logisticsdf[logisticsdf.asset=='Last Mile']
merged_lastmile = pd.merge(lastmile_df[~lastmile_df.orphan_id.isna()], rcdf[~rcdf.orphan_id.isna()][['orphan_id']], left_on='orphan_id', right_on = 'orphan_id', how='left')
merged_lastmile['age']=0
merged_lastmile['age'] = (datetime.today().date() - pd.to_datetime(merged_lastmile['scanned_date']).dt.date)/np.timedelta64(1, 'D')

In [14]:
merged_lastmile['ageing_category'] = merged_lastmile['age'].apply(lambda x: '1-9 days' if x <= 9 else ('10-16 days' if ((x >9) & (x <= 16) ) else ('17-23') if ((x >16) & (x <= 23) ) else ('24-31' if ((x >23) & (x <= 31) ) else 'Older 31 days')))
pd.DataFrame(pd.pivot_table(merged_lastmile[~merged_lastmile.orphan_id.isna()], index=['ageing_category','asset'], values=['scanned_timestamp'], aggfunc=len)).reset_index().rename(columns={'scanned_timestamp': 'orphan_count'})

,ageing_category,asset,orphan_count
0,1-9 days,Last Mile,1933
1,10-16 days,Last Mile,2062
2,17-23,Last Mile,1447
3,24-31,Last Mile,1630
4,Older 31 days,Last Mile,6170


In [8]:
merged_lastmile.columns

Index(['scanned_timestamp', 'email_address', 'zone', 'asset_x', 'hub_name',
       'orphan_type', 'orphan_id', 'super_category', 'product_title_x',
       'product_value', 'scanned_date_x', 'weekend_x', 'month_x', 'year_x',
       'month_year_x', 'weeknum_x', 'rc_received_timestamp', 'received_by',
       'rc_name', 'orphan_id_condition', 'fsn_identified_warehouse', 'wsn_id',
       'final_area_rc', 'business_unit', 'supercategory', 'expiry_date',
       'product_title_y', 'packaging_condition', 'asset_y',
       'physical_segregation', 'quantity', 'scan_box_id', 'scanned_date_y',
       'weekend_y', 'month_y', 'year_y', 'month_year_y', 'weeknum_y', 'age'],
      dtype='object')

In [ ]:
orphan_columns

In [ ]:
# if date_agg=='month_yr':
table_columns = [x for x in month_year if x in overall_orphans_generated.columns]
table_columns.insert(0,'asset')
# print(table_columns)
overall_orphans_generated[table_columns]
print(table_columns)

In [ ]:
overall_orphans_generated.columns.tolist().sort(key=month_year)

In [ ]:
table_columns